In [24]:
import pandas as pd
import re

In [25]:
# --- 1. CARREGAR E SEPARAR OS DATASETS ---

# Carregar o dataset completo e os IDs já anotados
#df_full = pd.read_csv('/Users/giossaurus/Developer/leia_tcc/data/processed/amostra_trabalho_1000.csv') # Usando a amostra de 1000 para teste rápido
# Para rodar em tudo, use: 
df_full = pd.read_csv('/Users/giossaurus/Developer/leia_tcc/data/interim/simulados_enem_completo.csv')

# Verificar colunas disponíveis
print(f"Colunas originais: {df_full.columns.tolist()}")
print(f"Primeiras linhas:")
print(df_full.head())

df_annotated_seed = pd.read_csv('/Users/giossaurus/Developer/leia_tcc/data/processed/seed_set_anotado.csv')
# Adicione os outros arquivos anotados se tiver
df_cycle1 = pd.read_csv('/Users/giossaurus/Developer/leia_tcc/data/processed/active_learning_cycle1.csv')
df_cycle2 = pd.read_csv('/Users/giossaurus/Developer/leia_tcc/data/processed/active_learning_cycle2.csv')

# Juntar todos os IDs já anotados
annotated_ids = df_annotated_seed['question_id'].unique()

# Separar nosso Golden Set de avaliação e o dataset de treino
df_golden_eval = df_full[df_full['question_id'].isin(annotated_ids)].copy()
df_weak_supervision_train = df_full[~df_full['question_id'].isin(annotated_ids)].copy()

print(f"Golden Set para avaliação: {len(df_golden_eval)} exemplos")
print(f"Dataset para Supervisão Fraca: {len(df_weak_supervision_train)} exemplos")
print(f"Colunas no dataset de treino: {df_weak_supervision_train.columns.tolist()}")

Colunas originais: ['question', 'true_answer', 'question_id', 'dataset', 'split']
Primeiras linhas:
                                            question  \
0  TEXTO DE REFERÊNCIA: Nos desfiles das escolas ...   
1  TEXTO DE REFERÊNCIA: A companhia de energia el...   
2  TEXTO DE REFERÊNCIA: A forma mais simples e an...   
3  TEXTO DE REFERÊNCIA: Cingapura é universalment...   
4  Texto de referência: Invocação de São Mateus (...   

                                         true_answer  \
0                                              36 m²   
1                                    V(x) = 3x – 288   
2                                hidratação do eteno   
3                         projeção no mercado global   
4  Estilo religioso e dramático, com clara demons...   

                                         question_id               dataset  \
0  512bec889180958c6b2e1c395931312ecc916a2039930d...            Matemática   
1  15da8d828a79fd53c164ac141d0ff2da49b16ade1618ae...            Matemá

In [26]:
# --- 2. DEFINIR AS FUNÇÕES DE ROTULAÇÃO (LFs) - VERSÃO 2 ---

# Mapear rótulos para constantes numéricas para facilitar
CONCEITUAL = 0
PROCEDIMENTAL = 1
ANALISE_EXEMPLO = 2
COMPARATIVO = 3
ABSTAIN = -1 # Abster-se de rotular

def lf_conceitual_keywords(text):
    # Mantivemos as mesmas, são boas
    keywords = ["o que é", "defina", "explique", "qual a função", "se refere a", "caracteriza-se por"]
    if any(kw in text.lower() for kw in keywords):
        return CONCEITUAL
    return ABSTAIN

def lf_procedimental_keywords(text):
    # Adicionamos muitas palavras-chave novas e mais comuns em matemática/física/química
    keywords = ["calcule", "determine", "qual o valor", "o resultado da", "a solução para", "encontre a"]
    if any(kw in text.lower() for kw in keywords):
        return PROCEDIMENTAL
    return ABSTAIN
    
def lf_comparativo_keywords(text):
    # Adicionamos algumas variações
    keywords = ["diferença entre", "compare", "em relação a", "distingue", "enquanto o outro", "diferentemente de"]
    if any(kw in text.lower() for kw in keywords):
        return COMPARATIVO
    return ABSTAIN

def lf_analise_exemplo_structure(text):
    # Tornamos a regra mais geral para pegar mais exemplos
    keywords = ["segundo o texto", "de acordo com o texto", "a charge", "o gráfico", "a tabela", "o autor do texto", "no trecho"]
    if any(kw in text.lower() for kw in keywords):
        return ANALISE_EXEMPLO
    return ABSTAIN

def lf_matematica_procedimental(text):
    # Função robusta que funciona sem depender de colunas específicas
    # Identifica questões matemáticas procedimentais por palavras-chave específicas
    math_keywords = ['equação', 'gráfico', 'função', 'área', 'volume', 'probabilidade', 'expressão', 
                     'calcule', 'determine', 'resolva', 'raiz', 'derivada', 'integral', 'matriz',
                     'logaritmo', 'sen', 'cos', 'tan', 'x =', 'y =', 'f(x)', 'solução']
    if any(kw in text.lower() for kw in math_keywords):
        return PROCEDIMENTAL
    return ABSTAIN

# Lista de todas as nossas funções de rotulação
labeling_functions = [
    lf_conceitual_keywords,
    lf_procedimental_keywords,
    lf_comparativo_keywords,
    lf_analise_exemplo_structure,
    lf_matematica_procedimental
]

In [27]:
# --- 3. APLICAR AS LFs E RESOLVER CONFLITOS ---

# Agora todas as funções usam apenas texto, então podemos aplicar todas de forma consistente
for lf in labeling_functions:
    df_weak_supervision_train[lf.__name__] = df_weak_supervision_train['question'].apply(lf)

# Pegar apenas as colunas de rótulos
lf_labels = df_weak_supervision_train[[lf.__name__ for lf in labeling_functions]]

# Resolver conflitos com voto majoritário
def majority_vote(row):
    votes = [label for label in row if label != ABSTAIN]
    if not votes:
        return ABSTAIN
    # Retorna o voto mais frequente
    return max(set(votes), key=votes.count)

df_weak_supervision_train['weak_label'] = lf_labels.apply(majority_vote, axis=1)

# Filtrar apenas os exemplos que conseguimos rotular
df_final_train_ws = df_weak_supervision_train[df_weak_supervision_train['weak_label'] != ABSTAIN].copy()

print(f"\nConseguimos gerar rótulos fracos para {len(df_final_train_ws)} exemplos.")
print("Distribuição dos rótulos fracos:")
# Mapear os IDs de volta para texto para visualização
id2label_ws = {0: 'Conceitual', 1: 'Procedimental', 2: 'Análise de Exemplo', 3: 'Comparativo'}
print(df_final_train_ws['weak_label'].map(id2label_ws).value_counts())

# Mostrar alguns exemplos de cada categoria
print("\n--- EXEMPLOS DE CADA CATEGORIA ---")
for label_id, label_name in id2label_ws.items():
    examples = df_final_train_ws[df_final_train_ws['weak_label'] == label_id]['question'].head(2)
    if len(examples) > 0:
        print(f"\n{label_name.upper()}:")
        for i, example in enumerate(examples, 1):
            print(f"  {i}. {example[:100]}...")

# Salvar o dataset de treino para a próxima etapa
df_final_train_ws.to_csv('/Users/giossaurus/Developer/leia_tcc/data/processed/weak_supervision_train_set.csv', index=False)


Conseguimos gerar rótulos fracos para 5854 exemplos.
Distribuição dos rótulos fracos:
weak_label
Procedimental         5412
Conceitual             198
Análise de Exemplo     171
Comparativo             73
Name: count, dtype: int64

--- EXEMPLOS DE CADA CATEGORIA ---

CONCEITUAL:
  1. TEXTO DE REFERÊNCIA: As indústrias de galvanoplastia que realizam a deposição de níquel em suas peça...
  2. TEXTO DE REFERÊNCIA: A fibrose cística (FC) é uma condição hereditária grave. […] Caracteriza-se por...

PROCEDIMENTAL:
  1. TEXTO DE REFERÊNCIA: Nos desfiles das escolas de samba do Rio de Janeiro, é cada vez mais comum obse...
  2. TEXTO DE REFERÊNCIA: A companhia de energia elétrica de uma cidade realiza a cobrança do forneciment...

ANÁLISE DE EXEMPLO:
  1. TEXTO DE REFERÊNCIA: Antes do primeiro navio porta-contêiner começar a navegar, há cerca de 50 anos,...
  2. TEXTO DE REFERÊNCIA: Are female artists worth collecting?

Tate doesn’t seem to think so

The museum...

COMPARATIVO:
  1. TEXTO DE 